## Setup Environment

In [1]:
%%capture
!pip install langchain
!pip install langchain-hub
!pip install langchain-community langchain-huggingface
!pip install huggingface_hub transformers
!pip install sentence_transformers==2.2.2
!pip install chromadb faiss accelerate
!pip install -U bitsandbytes
!pip install tiktoken python-dotenv

## Import modules

In [2]:
# llm modules
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from transformers import GenerationConfig
from langchain.llms import HuggingFacePipeline

# chain
from langchain.chains import ConversationChain
from langchain.chains import LLMChain

# prompt modules
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

# base modules
import os
import torch
import warnings

warnings.filterwarnings("ignore")

## LLM

In [3]:
model_name = "minkhantycc/Llama-2-7b-chat-finetune-quantized"
device_map = {"": 0}

# bnb config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)


# base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model.generation_config = GenerationConfig(
    max_new_tokens = 256,
    temperature = 0.01,
    repetition_penalty = 1.15,
    do_sample = False,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.eos_token_id,
)

# pipeline
pipe = pipeline(
    task="text-generation",
    model=base_model,
    tokenizer=tokenizer,
    device_map=device_map,
    return_full_text=False
)

# llm
llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [27]:
# create a Chat Prompt template
template_string = """<s>[INST]<<SYS>> \
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its context. \
If the AI does not know the answer to a question, it truthfully says it does not know.<</SYS>>

Human: {input}
AI: [/INST]"""

prompt_template = PromptTemplate.from_template(template_string)
print(prompt_template)

input_variables=['input'] template='<s>[INST]<<SYS>> The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.<</SYS>>\n\nHuman: {input}\nAI: [/INST]'


## Chain

In [28]:
chain = LLMChain(llm=llm, prompt=prompt_template)

chain.run("What is AI?")

"AI stands for Artificial Intelligence. It refers to any computer system that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, making decisions based on data, etc. \n\nThere are many different types of AI systems, including machine learning algorithms, deep neural networks, natural language processing (NLP) models, expert systems, rule-based systems, and more. Each type has its own strengths and weaknesses, depending on what problem they're designed to solve. For example, NLP models are great at understanding text input but struggle with visual recognition; while deep neural networks excel at image recognition but need large amounts of training data to be effective. \n\nThe field of AI research is constantly evolving, with new breakthroughs being made every year. Some recent developments include advances in reinforcement learning, which allows AI agents to learn how to play complex games like Go or poker without ex

## Simple Sequential Chain

In [7]:
from langchain.chains import SimpleSequentialChain

In [29]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<<SYS>>What is the best name to describe \
    a company that makes {product}?<</SYS>>[/INST]"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<</SYS>>Write a 20 words description for the following \
    company:{company_name}<</SYS>>[/INST]"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

overall_simple_chain.run("Shampoo")



> Entering new SimpleSequentialChain chain...
 The best name for a company that makes shampoos would be something like "Shine & Soft" or "Clean Hair Co.". These names are simple and easy to remember, while also conveying what the company does.

Alternatively, you could use a play on words such as "Hair-Tastic" or "Fresh Cuts". These names are catchy and memorable, making them ideal for branding purposes.

Ultimately, the best name will depend on your target audience and the tone of voice you want to convey through your branding efforts.
 Shine & Soft is a good name for a company that makes shampoo because it's short, easy to remember, and clearly communicates the product's purpose. It's also a bit more straightforward than some other options, which can make it easier to market and advertise the product effectively. However, if you want to add a bit more personality to your brand, you might consider using a play on words like "Hair-Tastic" or "Fresh Cuts", which have a fun and lighthe

' Shine & Soft is a good name for a company that makes shampoo because it\'s short, easy to remember, and clearly communicates the product\'s purpose. It\'s also a bit more straightforward than some other options, which can make it easier to market and advertise the product effectively. However, if you want to add a bit more personality to your brand, you might consider using a play on words like "Hair-Tastic" or "Fresh Cuts", which have a fun and lighthearted feel without being too cheesy or overly cute. Ultimately, the right name will depend on your specific goals and target audience, so take some time to think about how you want to position yourself in the market before settling on any particular option.'

## Sequential Chain

In [31]:
from langchain.chains import SequentialChain

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<<SYS>>Translate the following review to english:<</SYS>>"
    "\n\n{Review}[/INST]"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )

In [32]:
second_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<<SYS>>Can you summarize the following review in 1 sentence:<</SYS>>"
    "\n\n{English_Review}[/INST]"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [33]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<<SYS>>What language is the following review:\n\n<</SYS>>{Review}[/INST]"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [34]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "<s>[INST]<<SYS>>Write a follow up response to the following "
    "summary in the specified language:<</SYS>>"
    "\n\nSummary: {summary}\n\nLanguage: {language}[/INST]"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [35]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [36]:
review = (
    "Crafting a budget-friendly shoe that excels in multiple areas is challenging,"
    "yet we think Adidas has done a remarkable job with the Duramo Speed."
    "It offers a lightweight ride with superb breathability, complemented by a durable outsole and a solid Lightstrike midsole."
    "While it might be a bit stiffer than we'd like, and"
    "the 6.0-mm drop isn't for everyone, we believe it's an excellent choice for those seeking great value."
 )

response = overall_chain(review)
print(response)
print(type(response))



> Entering new SequentialChain chain...

> Finished chain.
{'Review': "Crafting a budget-friendly shoe that excels in multiple areas is challenging,yet we think Adidas has done a remarkable job with the Duramo Speed.It offers a lightweight ride with superb breathability, complemented by a durable outsole and a solid Lightstrike midsole.While it might be a bit stiffer than we'd like, andthe 6.0-mm drop isn't for everyone, we believe it's an excellent choice for those seeking great value.", 'English_Review': ' The Adidas Duramo Speed is a budget-friendly shoe that excels in several areas despite its affordability. It features a lightweight design with impressive breathability, thanks to a durable outsole and a high-quality midsole called Lightstrike. While some may find the shoe a bit too stiff or prefer a lower heel-to-toe drop (which this model does not offer), overall, it represents great value for runners on a budget. ', 'summary': ' Sure! Here\'s a summary of the review in one sen

## Router Chain

In [37]:
physics_template = """<s>[INST]<</SYS>>You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:<</SYS>>
{input}[/INST]"""


math_template = """<s>[INST]<<SYS>>You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:<</SYS>>
{input}[/INST]"""

history_template = """<s>[INST]<<SYS>>You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:<</SYS>>
{input}[/INST]"""


computerscience_template = """<s>[INST]<<SYS>>You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:<</SYS>>
{input}[/INST]"""

In [38]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [20]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [39]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [41]:
default_prompt = ChatPromptTemplate.from_template("<s>[INST]<<SYS>>{input}<</SYS>>[/INST]")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [42]:
MULTI_PROMPT_ROUTER_TEMPLATE = """<s>[INST] <<SYS>>Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}
<</SYS>>
<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>[/INST]"""

In [43]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [44]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is blackbody radiation?'}
> Finished chain.


' Blackbody radiation is electromagnetic radiation emitted by an idealized perfect absorber of electromagnetic radiation, known as a blackbody. The radiation is characterized by its temperature and wavelength, and it follows specific statistical distributions that depend on the temperature of the body. These distributions were first derived by Max Planck in 1900 and have since been widely used in many areas of science and engineering.\n\nBlackbody radiation has several important applications, including:\n\n* Thermal radiation: This type of radiation is emitted by objects at temperatures above absolute zero (0 Kelvin). It is often referred to as "heat" or "thermal energy."\n* Spectroscopy: This involves measuring the intensity of light emitted by an object at different wavelengths. By analyzing these spectra, scientists can learn more about the chemical composition of materials and their physical properties.\n* Quantum mechanics: Blackbody radiation plays a key role in understanding the

In [45]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Every cell in our body contains DNA because this molecule serves as the genetic material, carrying information about an organism’s traits and characteristics. This information allows cells to grow, divide, and function properly.'}
> Finished chain.


"  As a physicist, I must first acknowledge that my knowledge of biology is limited compared to other fields such as biochemistry or molecular biology. However, based on what I do know, it seems that every cell in our bodies does indeed contain DNA, which serves as the genetic material for those cells. The DNA carries information about the organism's traits and characteristics, allowing cells to grow, divide, and function properly. While there may be some exceptions, such as certain types of stem cells, most cells in the human body have DNA within their cytoplasm. "